In [20]:
import pickle

heterodata = pickle.load(open("hetero_graph_final.pkl", "rb"))

In [21]:
heterodata

HeteroData(
  paper={
    license=[1000],
    doi=[1000],
    pages=[1000],
    journal=[1000],
    date=[1000],
    id=[1000],
    name=[1000],
    num_nodes=1000,
  },
  author={
    name=[2886],
    num_nodes=2886,
  },
  category={
    name=[108],
    num_nodes=108,
  },
  word={
    name=[10667],
    num_nodes=10667,
  },
  journal={
    name=[574],
    num_nodes=574,
  },
  (paper, written_by, author)={ edge_index=[2, 3068] },
  (paper, has_category, category)={ edge_index=[2, 1502] },
  (paper, has_word, word)={
    edge_index=[2, 77703],
    edge_attr=[77703],
  },
  (paper, has_titleword, word)={ edge_index=[2, 7462] },
  (paper, in_journal, journal)={ edge_index=[2, 1000] },
  (word, co_occurs_with, word)={
    edge_index=[2, 499132],
    edge_attr=[499132],
  }
)

In [3]:
heterodata['paper']['license'] = [license if license is not None else 'None' for license in heterodata['paper']['license']]

In [4]:
# idx = 336
# idx2 = 4
# a= heterodata[heterodata.edge_types[idx2]].edge_index
# from_node = heterodata[heterodata.edge_types[idx2][0]].name[a[0,idx]]
# to_node = heterodata[heterodata.edge_types[idx2][2]].name[a[1,idx]]

# print(heterodata.edge_types[idx2][0], from_node)
# print(heterodata.edge_types[idx2][2], to_node)

In [5]:
# heterodata[('paper', 'in_journal', 'journal')] = heterodata[('paper', 'in_journal', 'journal-ref')]
# del heterodata[('paper', 'in_journal', 'journal-ref')]

# remove isolated nodes
import torch_geometric.transforms as T
transform = T.Compose([
       #T.RemoveIsolatedNodes(),
       T.RemoveDuplicatedEdges(),
       # T.ToUndirected(merge=False) # don't merge reversed edges into the original edge type
])
from torch_geometric.data import HeteroData 

# from copy import deepcopy
# data = deepcopy(heterodata)
# for node_type in heterodata.node_types[-5:]:
       # heterodata[node_type].num_nodes = hetero

# for edge_type in  heterodata.edge_types[-:]:
       # print(edge_type)
       # 
       # del data[edge_type]
       # del data[list(reversed(heterodata.edge_types))[5]]

# for node_type in heterodata.node_types[:6]:
#        print(node_type)
#        del data[node_type]
# test = transform(data)
heterodata = transform(heterodata)

In [6]:
import torch
# id mapping so we can remove the attributes and then remove isolated nodes, later we can add the attributes back
id_dict = {}
heterodata_dict = heterodata.to_dict()
for nodetype in heterodata.node_types:
    print(nodetype)
    ids = torch.arange(heterodata[nodetype].num_nodes) 
    id_mapping = {i.item():name for i, name in zip(ids, heterodata[nodetype].name)}
    
    heterodata[nodetype].x = ids.squeeze(-1)
    id_dict[nodetype] = id_mapping
    
    for key in heterodata_dict[nodetype].keys():
        if key != 'x':
            print('del', nodetype, key)
            del heterodata[nodetype][key]

paper
del paper license
del paper doi
del paper pages
del paper journal
del paper date
del paper id
del paper name
del paper num_nodes
author
del author name
del author num_nodes
category
del category name
del category num_nodes
word
del word name
del word num_nodes
journal
del journal name
del journal num_nodes


In [7]:
heterodata

HeteroData(
  paper={ x=[1000] },
  author={ x=[2886] },
  category={ x=[108] },
  word={ x=[10667] },
  journal={ x=[574] },
  (paper, written_by, author)={ edge_index=[2, 3067] },
  (paper, has_category, category)={ edge_index=[2, 1502] },
  (paper, has_word, word)={
    edge_index=[2, 57073],
    edge_attr=[57073],
  },
  (paper, has_titleword, word)={ edge_index=[2, 7354] },
  (paper, in_journal, journal)={ edge_index=[2, 1000] },
  (word, co_occurs_with, word)={
    edge_index=[2, 499132],
    edge_attr=[499132],
  }
)

In [8]:
# check correct edge types:
transform = T.Compose([
       T.RemoveIsolatedNodes(),
       #    T.RemoveDuplicatedEdges(),
       # T.ToUndirected(merge=False) # don't merge reversed edges into the original edge type
])

heterodata = transform(heterodata)

In [9]:
heterodata


HeteroData(
  paper={ x=[1000] },
  author={ x=[2886] },
  category={ x=[108] },
  word={ x=[10604] },
  journal={ x=[574] },
  (paper, written_by, author)={ edge_index=[2, 3067] },
  (paper, has_category, category)={ edge_index=[2, 1502] },
  (paper, has_word, word)={
    edge_index=[2, 57073],
    edge_attr=[57073],
  },
  (paper, has_titleword, word)={ edge_index=[2, 7354] },
  (paper, in_journal, journal)={ edge_index=[2, 1000] },
  (word, co_occurs_with, word)={
    edge_index=[2, 499132],
    edge_attr=[499132],
  }
)

In [10]:
# from tqdm.auto import tqdm
# import json 
# import pandas as pd 

# def read_first_n_lines(file_path, n=10000):
#     data = []
#     with open(file_path, 'r') as file:
#         for i, line in tqdm(enumerate(file), total=n):
#             if i >= n:
#                 break
#             try:
#                 data.append(json.loads(line))
#             except json.JSONDecodeError:
#                 continue
#     return pd.DataFrame(data)

# file_path = '../make_the_graph/data/arxiv-metadata-oai-snapshot.json'

# # DataFrame erstellen
# df = read_first_n_lines(file_path, 1000)

In [11]:
heterodata = T.ToUndirected()(heterodata)


In [12]:
# map back
for nodetype in heterodata.node_types:
    print(nodetype)
    id_mapping = id_dict[nodetype]
    heterodata[nodetype].name = [id_mapping[i.item()] for i in heterodata[nodetype].x]
    del heterodata[nodetype].x

paper
author
category
word
journal


In [13]:
# idx = 330
# idx2 = 4
# a= heterodata[heterodata.edge_types[idx2]].edge_index
# from_node = heterodata[heterodata.edge_types[idx2][0]].name[a[0,idx]]
# to_node = heterodata[heterodata.edge_types[idx2][2]].name[a[1,idx]]

# print(heterodata.edge_types[idx2][0], from_node)
# print(heterodata.edge_types[idx2][2], to_node)

In [14]:
heterodata

HeteroData(
  paper={ name=[1000] },
  author={ name=[2886] },
  category={ name=[108] },
  word={ name=[10604] },
  journal={ name=[574] },
  (paper, written_by, author)={ edge_index=[2, 3067] },
  (paper, has_category, category)={ edge_index=[2, 1502] },
  (paper, has_word, word)={
    edge_index=[2, 57073],
    edge_attr=[57073],
  },
  (paper, has_titleword, word)={ edge_index=[2, 7354] },
  (paper, in_journal, journal)={ edge_index=[2, 1000] },
  (word, co_occurs_with, word)={
    edge_index=[2, 871906],
    edge_attr=[871906],
  },
  (author, rev_written_by, paper)={ edge_index=[2, 3067] },
  (category, rev_has_category, paper)={ edge_index=[2, 1502] },
  (word, rev_has_word, paper)={
    edge_index=[2, 57073],
    edge_attr=[57073],
  },
  (word, rev_has_titleword, paper)={ edge_index=[2, 7354] },
  (journal, rev_in_journal, paper)={ edge_index=[2, 1000] }
)

In [ ]:
# save for training
import pickle
pickle.dump(heterodata, open("arxiv_author_paper_graph_no_features.pkl", "wb"))


In [ ]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
# for nodetype in heterodata.node_types:
#     print(nodetype, heterodata[nodetype].x.shape, heterodata[nodetype].num_nodes)

In [16]:
def describe(heterodata):
    heterodata_dict = heterodata.to_dict()
    # print('has isolated nodes:', heterodata.has_isolated_nodes())
    # print('has self loops:', heterodata.has_self_loops())
    # print('is directed', heterodata.is_directed())
    for nodetype in heterodata.node_types:
        print(nodetype)
        print('  ',heterodata[nodetype].num_nodes)
        print('  ',heterodata_dict[nodetype].keys())
        
    for edgetype in heterodata.edge_types:
        print(edgetype)
        print('  ',heterodata[edgetype].num_edges)
        print('  ',heterodata[edgetype])

# describe(heterodata)

paper
   None
   dict_keys(['name'])
author
   None
   dict_keys(['name'])
category
   None
   dict_keys(['name'])
word
   None
   dict_keys(['name'])
journal
   None
   dict_keys(['name'])
('paper', 'written_by', 'author')
   3067
   {'edge_index': tensor([[   0,    0,    0,  ...,  997,  998,  999],
        [   0,    1,    2,  ..., 2883, 2884, 2885]])}
('paper', 'has_category', 'category')
   1502
   {'edge_index': tensor([[  0,   1,   1,  ..., 997, 998, 999],
        [  0,   1,   2,  ...,  10,  52,  15]])}
('paper', 'has_word', 'word')
   57073
   {'edge_index': tensor([[    0,     0,     0,  ...,   999,   999,   999],
        [    0,     1,     2,  ..., 10209, 10210, 10211]]), 'edge_attr': tensor([0.0942, 0.0789, 0.2729,  ..., 0.1235, 0.1235, 0.1235])}
('paper', 'has_titleword', 'word')
   7354
   {'edge_index': tensor([[    0,     0,     0,  ...,   999,   999,   999],
        [    2,     7,    41,  ..., 10191, 10192, 10193]])}
('paper', 'in_journal', 'journal')
   1000
   {'edge_in

/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'name'}'. Please explicitly set 'num_nodes' as an attribute of 'data[paper]' to suppress this warning
  warnings.warn(
/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'name'}'. Please explicitly set 'num_nodes' as an attribute of 'data[author]' to suppress this warning
  warnings.warn(
/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'name'}'. Please explicitly set 'num_nodes' as an attribute of 'data[category]' to suppress this warning
  warnings.warn(
/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/data/storage.py:327: UserWarning: Una